# Introduction to Smartnoise-SQL

[Smartnoise-SQL](https://docs.smartnoise.org/sql/index.html) is a python library that enables to perform differentially private SQL queries. 

SmartNoise is intended for scenarios where the analyst is trusted by the data owner.

## Step 1: Install the Library

Smartnoise-sql is available on pypi, it can be installed via the pip command. We will use the latest version of the library to date: version 1.0.6.

In [2]:
!pip install smartnoise-sql==1.0.6

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 17.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144592 sha256=0c710416fbf6c163536b091fa7ff2970a6e6e495c5c86fc484a0903cbe792c6a
  Stored in directory: /home/bfs/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime


## Step 2: Load and Prepare Data

In this notebook, we will work with the [penguin dataset]("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv") from [seaborn datasets](https://github.com/mwaskom/seaborn-data).
We load the dataset via pandas in a dataframe `df`.

In [1]:
import pandas as pd

In [2]:
path_to_data = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
df = pd.read_csv(path_to_data)

We can look at the first rows of the dataframe to get to know the data:

In [3]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


We see that there are 7 columns: 'species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g' and 'sex' with various data types.

## Step 3: Prepare Analysis with Smartnoise-SQL

Before doing a query, `smartnoise-sql` requires a reader object [(see doc here)](https://docs.smartnoise.org/sql/api/index.html#snsql.connect.from_df). When working with pandas dataframe, this object takes three parameters:
- df: The Pandas DataFrame to be queried (which we loaded in step 2)
- privacy: A Privacy object with the desired privacy parameters (we instantiate it in step 3.1)
- metadata: The metadata describing the data source (we instantiate it in step 3.2)

### Step 3.1: Privacy object

The `Privacy` object [(see doc)](https://docs.smartnoise.org/sql/api/index.html#privacy) enables to select the privacy budget used by queries. The budget is specified with an $\epsilon$ and $\delta$ as in approximate differential privacy.
$$
Pr[M(S) \in O] < e^{\epsilon} Pr[M(S') \in O] + \delta
$$

We select $\epsilon=0.1$ and $\delta=0.0001$.

**Note**: Each query will be decomposed in 'sub-queries' by `smartnoise-sql`, which will all be excuted with DP noise and then assembled to get a result. The budget specified here is the budget spent by each sub-query (and not the total budget of the query). For instance for an average query, there will be two sub-queries: a sum and a count. Each will cost the $\epsilon$ and $\delta$ provided and return a result (so the double of what is given in input is spent). Then `smartnoise-sql` returns the result $dp-mean=\frac{dp-sum}{dp-count}$. The advanced exercise in step 6 gives an example of this mechanism.

Optionnaly, it can also represent desired accuracy bounds or specify mechanisms for certain statistics but this is out of scope for this notebook.

In [4]:
from snsql import Privacy

In [5]:
# TODO: fill epsilon and delta values
# EPSILON = ...
# DETLA = ...

# Correction
EPSILON = 0.1
DELTA = 1/10000

In [6]:
privacy = Privacy(epsilon=EPSILON, delta=DELTA)

### Step 3.2: Prepare the metadata

Next we prepare the metadata. The format expected is explained [here](https://docs.smartnoise.org/sql/metadata.html#metadata) in `smartnoise-sql` documentation. It can be provided in different format such as an external `yaml` file or a dictionnary. In this notebook we will use the [dictionnary format](https://docs.smartnoise.org/sql/metadata.html#dictionary-format).

There are `Table Options` and `Column Options`. 
- `Table Options` apply on the whole table and can further configure queries. The have predertermined default values and should only be overriden with caution.
    - `max_ids` specifies in how many rows each unique user can appear in. In our case, each penguin appear only once, hence `max_ids=1`.
    - `row_privacy` tells the system to treat each row as being a single individual. It is True in our case. Each row is a penguin.
    - `censor_dims` drops GROUP BY output rows that might reveal the presence of individuals in the database. As the dimensions (available species, island and sex) are public, we set this to false.
    - We will keep the default values for all other fields.
- `Column Options` are compulsory and describe the table column by column.
    - Each column must have the exact same name in the metadata and in the column of the table.
    - Each column needs a `type`, indicates the type for all values in the column (type may be one of 'int', 'float', 'string', 'boolean', or 'datetime').
    - Columns with numbers ('int', 'float') should additionally have `lower` and `upper` bounds. Meaning the minimum and maximum theoretically possible values for this column. This is important as it enables to compute the sensitivy and hence, calibrate the differentially private noise.
    - Optionnally a boolean `nullable` can be provided if the user knows that there are no `null` values. By default it is True meaning that columns may contain `null` values.
    - Other options are possible but won't be treated in this notebook.

For `Table Options`, we only specify `max_ids` (and keep all other the default values too).

In [7]:
table_options = {
    'max_ids': 1,
    'row_privacy': True,
    'censor_dims': False
}

For `Column Options`, We look at the dataset again to determine the types:

In [8]:
df.head(1)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE


In [9]:
species_col = {'type': 'string', 'nullable': False}
island_col = {'type': 'string', 'nullable': False}
bill_length_col = {'type': 'float', 'lower': 30.0, 'upper': 65.0}
bill_depth_col = {'type': 'float', 'lower': 13.0, 'upper': 23.0}
flipper_length_col = {'type': 'float', 'lower': 150.0, 'upper': 250.0}

In [10]:
# TODO: Fill body_mass_g and sex column metadata knowing that these specied of penguins typically weight between 2000.0 and 7000.0 grammes. We cannot say for sure that there are no nulls in these columns.
# body_mass_g_col = ...
# sex_col = ...

# Correction
body_mass_g_col = {'type': 'float', 'lower': 2000.0, 'upper': 7000.0}
sex_col = {'type': 'string'}

In [11]:
# 'str' is for a chain of character and 'float' is for decimal numbers.
columns_options = {
    'species': species_col,
    'island': island_col,
    'bill_length_mm': bill_length_col,
    'bill_depth_mm': bill_depth_col,
    'flipper_length_mm': flipper_length_col,
    'body_mass_g': body_mass_g_col,
    'sex': sex_col, 
}

Then, we assemble the metadata respecting the format expected by `smartnoise-sql`. As it can work on multiple table, it needs some additional outer fields.

In [12]:
# Build metadata
metadata = {
    '': {
        '': {
            'penguin_table': {
                **table_options,
                **columns_options
            }
        }
    }
}

In [13]:
metadata

{'': {'': {'penguin_table': {'max_ids': 1,
    'row_privacy': True,
    'censor_dims': False,
    'species': {'type': 'string', 'nullable': False},
    'island': {'type': 'string', 'nullable': False},
    'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
    'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
    'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
    'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
    'sex': {'type': 'string'}}}}}

### Step 3.3: Instantiate the reader

All arguments are now available to create the reader object mentionned at the begin of step 3.

In [14]:
from snsql import from_df

In [15]:
# TODO: Instantiate the reader object
# reader = ...

# Correction
reader = from_df(df, metadata=metadata, privacy=privacy)

## Step 4: Differentially Private Dataset Query

We now only miss the actual query to execute. Note: the table was named `penguin_table` in the metadata, so the queries must always query `FROM penguin_table`.

### Count Number of Penguins
Let's write one to count the number of rows in the dataset:

In [16]:
COUNT_QUERY = "SELECT COUNT(*) AS nb_penguins FROM penguin_table"

Depending on the query, the cost spent by the query might be different than the one
We can first verify the privacy cost of this query:

In [17]:
epsilon, delta = reader.get_privacy_cost(COUNT_QUERY)
print(f"This query will cost {epsilon} epsilon and {delta} delta.")

This query will cost 0.1 epsilon and 0.00014999500000001387 delta.


And then execute it:

In [18]:
result = reader.execute(COUNT_QUERY)
result

[['nb_penguins'], [347]]

It is not very far from the true result:

In [19]:
print(f"Non-DP number of penguins: {df.shape[0]}.")
print(f"DP number of penguins: {result[1][0]}.")
print(f"Difference: {abs(df.shape[0] - result[1][0])}.")

Non-DP number of penguins: 344.
DP number of penguins: 347.
Difference: 3.


### Average Bill Length of Penguins
Now write and experiment with an average query on the bill length.

In [20]:
# TODO: write a query to compute the average bill length, check its budget and execute it
# AVG_QUERY = ...
# epsilon, delta = ...
# result = ...

# Correction
AVG_QUERY = "SELECT AVG(bill_length_mm) AS avg_bill_length_mm FROM penguin_table"
epsilon, delta = reader.get_privacy_cost(AVG_QUERY)
result = reader.execute(AVG_QUERY)

print(f"This query will cost {epsilon} epsilon and {delta} delta.")
print(f"DP average bill length: {result[1][0]}mm.")

This query will cost 0.2 epsilon and 4.999999999999449e-05 delta.
DP average bill length: 41.50016776429587mm.


### Average Bill Length by Species of Penguins

Now write a query to compute the average bill length per species:

In [39]:
# TODO: write a query to compute the average bill length of penguin per species
#AVG_GROUP_QUERY = """
#    SELECT 
#    species AS species, \
#    ... AS avg_bill_length_mm \
#    FROM ... \
#    GROUP BY ...
#"""

# Correction
AVG_GROUP_QUERY = """
    SELECT 
    species AS species,
    AVG(bill_length_mm) AS avg_bill_length_mm
    FROM penguin_table
    GROUP BY species
"""

And verify the privacy budget of the query. Note: can you guess what it is before executing the cell ?

In [40]:
epsilon, delta = reader.get_privacy_cost(AVG_GROUP_QUERY)
print(f"This query will cost {epsilon} epsilon and {delta} delta.")

This query will cost 2.0 epsilon and 0.050000000000000044 delta.


Now, we execute it:

In [23]:
avg_result = reader.execute(AVG_GROUP_QUERY)

In [24]:
def format_results(results):
    """Small function to format results in pd.DataFrame"""
    result = results[:]
    cols = result.pop(0)
    return pd.DataFrame(result, columns=cols)

In [25]:
avg_result = format_results(avg_result)
avg_result

,species,avg_bill_length_mm
0,Adelie,36.479662
1,Chinstrap,70.175246
2,Gentoo,55.870861


## Step 5: Learn How to Use the Odometer

When working with differentially private queries, there are two common approaches to tracking privacy loss budget spending.

1. **Summing the privacy cost of each individual query manually** – i.e., calculating epsilon and delta for one query and adding it for each executions.  
2. **Using the privacy odometer** – a mechanism provided by the `PrivateReader` that tracks accumulated privacy spending automatically as queries are executed.

We will show the two versions here. First, we initialise the reader and check that the privacy spent so far is $0$.

In [52]:
privacy = Privacy(epsilon=0.1, delta=10e-7)
reader = from_df(df, metadata=metadata, privacy=privacy)
reader.odometer.spent

(0.0, 0.0)

We execute one query and check the cost of a single query with `get_privacy_cost` and with the odometer.

In [53]:
# Execute one query
query = 'SELECT AVG(bill_length_mm), AVG(flipper_length_mm) FROM penguin_table'
result = reader.execute(query)

# Privacy loss budget of one query with the odometer
epsilon_odometer, delta_odometer = reader.odometer.spent
print(f"Privacy loss budget of one query with the odometer ({epsilon_odometer}, {delta_odometer}).")

# Privacy loss budget of one query
epsilon_single, delta_single = reader.get_privacy_cost(query)
print(f"Privacy loss budget of one query (no odometer) ({epsilon_single}, {delta_single}).")

Privacy loss budget of one query with the odometer (0.4, 4.999999999588667e-07).
Privacy loss budget of one query (no odometer) (0.4, 4.999999999588667e-07).


We see that they are the same. Now let's compare the privacy loss budget of a hundred queries:

In [54]:
for _ in range(99):
    reader.execute(query)

epsilon_odometer_many, delta_odometer_many = reader.odometer.spent
print(f"Privacy loss budget of 100 queries with the odometer ({epsilon_odometer_many}, {delta_odometer_many}).")
print(f"Privacy loss budget of 100 queries (no odometer) ({epsilon_single * 100}, {delta_single * 100}).")

Privacy loss budget of 100 queries with the odometer (12.771879536126042, 4.999999999588667e-07).
Privacy loss budget of 100 queries (no odometer) (40.0, 4.999999999588667e-05).


Question: When doing multiple queries, is it better to use the odometer or is it better to simply add the cost of each individual query ? Why ?

**Short Answer**: The odometer because less budget is spent.

**Long Answer**: While multiplying the cost of a single query by the number of executions may seem straightforward, it assumes a worst-case additive composition of privacy. This approach can significantly overstate the true privacy loss, especially when queries are correlated or when advanced composition theorems apply.  

The odometer, on the other hand, tracks the actual accumulated privacy spending using proper composition rules. This means it can leverage advanced composition, parallel composition, and other optimizations built into the SmartNoise framework. 

In short, the odometer is better than simply adding up individual query costs because it correctly accounts for privacy composition across multiple queries and provides a tighter, more accurate estimate of total privacy consumption.  

## Step 6: Advanced Exercise with Hypothesis testing

We want to test if bill length differ between the penguins species with a two-tailed two-sample $t$ test and a level of significance at 0.05.
- The null hypothese $H_0$ is bill length does not differ between species.
- The alternative hypothesis $H_a$ is bill length does differ between species.

Therefore, we need 
- the number of penguin per species,
- the average bill length per species and,
- the standard deviation of bill length per species.

We first allow more budget for this query a create new the `privacy` and `reader` objects.

In [41]:
privacy = Privacy(epsilon=1.0, delta=0.1)
reader = from_df(df, metadata=metadata, privacy=privacy)

### Write query

Now write a query to get the summary statistics:

In [42]:
# TODO: Write a query that gets the number of penguin, average and standard deviation of bill length per penguin species
#HYP_QUERY = """
#    SELECT
#    species AS species,
#    ... AS nb_penguin,
#    ... AS avg_bill_length_mm,
#    ... AS std_bill_length_mm
#    FROM ...
#    GROUP BY ...
#"""

# Correction
HYP_QUERY = """
    SELECT
    species AS species,
    COUNT(bill_length_mm) AS nb_penguin,
    AVG(bill_length_mm) AS avg_bill_length_mm,
    STD(bill_length_mm) AS std_bill_length_mm
    FROM penguin_table
    GROUP BY species
"""

### Understand budget

Do you remember the command to estimate the budget of this query?

In [43]:
# TODO: Estimate budget of query with smartnoise_sql
#epsilon, delta = ...

# Correction
epsilon, delta = reader.get_privacy_cost(HYP_QUERY)
epsilon, delta

(np.float64(3.0), np.float64(0.050000000000000044))

We see that the privacy loss budget spent in epsilon is three times the input budget epsilon. Let's look into into smartnoise-sql to understand why (this is not necessary to use the library but unables us to understand how it works:

In [46]:
subquery, query = reader._rewrite(HYP_QUERY)
print("Outer query:", query)
print("")
print("Subquery:", subquery)

Outer query: SELECT species AS species, ( count_bill_length_mm ) AS nb_penguin, ( ( sum_bill_length_mm / count_bill_length_mm ) ) AS avg_bill_length_mm, ( SQRT ( ( sum_alias_0x708 / count_bill_length_mm ) - ( sum_bill_length_mm / count_bill_length_mm ) * ( sum_bill_length_mm / count_bill_length_mm ) ) ) AS std_bill_length_mm FROM ( SELECT species AS species, COUNT(bill_length_mm) AS count_bill_length_mm, SUM(bill_length_mm) AS sum_bill_length_mm, SUM(bill_length_mm * bill_length_mm) AS sum_alias_0x708 FROM ( SELECT species AS species, CASE WHEN bill_length_mm < 30.0 THEN 30.0 WHEN bill_length_mm > 65.0 THEN 65.0 ELSE  bill_length_mm END AS bill_length_mm FROM penguin_table ) AS per_key_all GROUP BY species ) AS exact_aggregates

Subquery: SELECT species AS species, COUNT(bill_length_mm) AS count_bill_length_mm, SUM(bill_length_mm) AS sum_bill_length_mm, SUM(bill_length_mm * bill_length_mm) AS sum_alias_0x708 FROM ( SELECT species AS species, CASE WHEN bill_length_mm < 30.0 THEN 30.0 WH

What happened is that smartnoise sql rewrote the query as three subqueries (without counting the SELECT on species):
- SELECT species AS species,
- COUNT(bill_length_mm) AS count_bill_length_mm,
- SUM(bill_length_mm) AS sum_bill_length_mm,
- SUM(bill_length_mm * bill_length_mm) AS sum_alias_0xc15f

And then postprocessed them with:
- SELECT species AS species,
- ( count_bill_length_mm ) AS nb_penguin,
- ( ( sum_bill_length_mm / count_bill_length_mm ) ) AS avg_bill_length_mm,
- ( SQRT ( ( sum_alias_0xc15f / count_bill_length_mm ) - ( sum_bill_length_mm / count_bill_length_mm ) * ( sum_bill_length_mm / count_bill_length_mm ) ) ) AS std_bill_length_mm

The get the result of the query. 

Each subquery cost the epsilon given in input. Therefore, the total is three times the input budget.

### Execute query

We execute the query below. If there is a `nan` in the output, we execute the query again.

In [44]:
hyp_result = reader.execute(HYP_QUERY)
df_result = format_results(hyp_result)

while df_result.isna().any().any():
    hyp_result = reader.execute(HYP_QUERY)
    df_result = format_results(hyp_result)
df_result

/home/bfs/.local/lib/python3.11/site-packages/snsql/_ast/expressions/numeric.py:144: RuntimeWarning: invalid value encountered in sqrt
  return funcs[self.name.lower()](exp)
/home/bfs/.local/lib/python3.11/site-packages/snsql/_ast/expressions/numeric.py:144: RuntimeWarning: invalid value encountered in sqrt
  return funcs[self.name.lower()](exp)
/home/bfs/.local/lib/python3.11/site-packages/snsql/_ast/expressions/numeric.py:144: RuntimeWarning: invalid value encountered in sqrt
  return funcs[self.name.lower()](exp)
/home/bfs/.local/lib/python3.11/site-packages/snsql/_ast/expressions/numeric.py:144: RuntimeWarning: invalid value encountered in sqrt
  return funcs[self.name.lower()](exp)
/home/bfs/.local/lib/python3.11/site-packages/snsql/_ast/expressions/numeric.py:144: RuntimeWarning: invalid value encountered in sqrt
  return funcs[self.name.lower()](exp)
/home/bfs/.local/lib/python3.11/site-packages/snsql/_ast/expressions/numeric.py:144: RuntimeWarning: invalid value encountered in 

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm
0,Adelie,151,38.065677,7.252814
1,Chinstrap,70,47.865236,9.467181
2,Gentoo,122,47.544631,7.568299


### Answer research question

And now we answer the research question with a t-test:

In [28]:
import numpy as np

In [29]:
def t_test(avg_1, avg_2, std_1, std_2, nb_1, nb_2):
    standard_error = (std_1 * (nb_1 - 1) + std_2 * (nb_2 - 1))/(nb_1 + nb_2 - 2)
    return (avg_1 - avg_2)/np.sqrt(standard_error**2*(1/nb_1 + 1 /nb_2))

In [30]:
nb_0, avg_0, std_0 = df_result[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[0]
nb_1, avg_1, std_1 = df_result[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[1]
nb_2, avg_2, std_2 = df_result[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[2]

In [31]:
t_01 = t_test(avg_0, avg_1, std_0, std_1, nb_0, nb_1)
t_02 = t_test(avg_0, avg_2, std_0, std_2, nb_0, nb_2)
t_12 = t_test(avg_1, avg_2, std_1, std_2, nb_1, nb_2)

CRITICAL_VALUE = 0.05
print(f"T test between species 0 and specie 1: {np.round(t_01, 2)}. Reject null hypothesis: {abs(t_01) > CRITICAL_VALUE}.")
print(f"T test between species 0 and specie 2: {np.round(t_02, 2)}. Reject null hypothesis: {abs(t_02) > CRITICAL_VALUE}.")
print(f"T test between species 1 and specie 2: {np.round(t_12, 2)}. Reject null hypothesis: {abs(t_12) > CRITICAL_VALUE}.")

T test between species 0 and specie 1: -1.93. Reject null hypothesis: True.
T test between species 0 and specie 2: 2.9. Reject null hypothesis: True.
T test between species 1 and specie 2: 3.52. Reject null hypothesis: True.


We also compute the confidence interval of bill length of each species:

In [32]:
ZSCORE = 1.96
df_result['standard_error'] = df_result['std_bill_length_mm']/np.sqrt(df_result['nb_penguin'])
df_result['ci_95_lower_bound'] = df_result['avg_bill_length_mm'] - ZSCORE * df_result['standard_error']
df_result['ci_95_upper_bound'] = df_result['avg_bill_length_mm'] + ZSCORE * df_result['standard_error']
df_result

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm,standard_error,ci_95_lower_bound,ci_95_upper_bound
0,Adelie,162,35.132618,26.661618,2.094735,31.026938,39.238298
1,Chinstrap,78,41.961487,23.510380,2.662026,36.743915,47.179058
2,Gentoo,122,22.955364,46.026458,4.167040,14.787966,31.122762
